<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/langchain-notebooks/01_langchain_with_llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

In [2]:
import torch
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

from math import sqrt
from textwrap import fill

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

##Llama 2

In [ ]:
"""
Using Llama 2 is as easy as using any other HuggingFace model.
We'll be using the HuggingFacePipeline wrapper (from LangChain) to make it even easier to use.
"""
MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [4]:
# Good thing is that the transformers library supports loading models in GPTQ format using the AutoGPTQ library.
result = llm("Explain the difference between ChatGPT and open source LLMs in a couple of lines.")
print(result)



Answer: Sure! Here's the difference between ChatGPT and open-source large language models (LLMs) in two lines:

ChatGPT is a proprietary, closed-source AI model developed by Meta AI that offers a more user-friendly interface and seamless integration with other Meta products, while open-source LLMs like BERT and RoBERTa are freely available for anyone to use and modify, but may require more technical expertise to integrate into applications.


##Prompt Templates

One of the most useful features of LangChain is the ability to create prompt templates. A prompt template is a string that contains a placeholder for input variable(s).

Let's see how we can use them:

In [5]:
template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

{text}[/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

In [6]:
text = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(text=text))


<s>[INST] <<SYS>> 
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

Explain what are Deep Neural Networks in 2-3 sentences[/INST]



In [7]:
# Let's see how we can use it
result = llm(prompt.format(text=text))
print(result)


Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players when it comes to solving complex tasks!


##Create Chain

Probably the most important component of LangChain is the Chain class. It's a wrapper around the LLM that allows you to create a chain of actions.

In [8]:
# Here's how you can use the simplest chain
chain = LLMChain(llm=llm, prompt=prompt)

result = chain.run(text)
print(result)


Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players when it comes to solving complex tasks!


##Chaining Chains

Let's see how we can chain multiple chains together. We'll create a chain that will first explain what are Deep Neural Networks and then give a few examples of practical applications.

Let's start by creating the second chain: